<a href="https://colab.research.google.com/github/Sina-Akhavi/bitcoin-timeseries-Forecasting/blob/main/cnn_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.chdir('./sample_data')

In [2]:
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
# ------------------------------------------------
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
# ------------------------------------------------
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten


maindf = pd.read_csv('./BTC-USD.csv')


In [3]:
print("Null values: ", maindf.isnull().values.sum())
print("NA values: ", maindf.isnull().values.any())

Null values:  0
NA values:  False


In [4]:
closedf = maindf[['Date', 'Close']]
closedf


,Date,Close
0,2014-09-17,457.334015
1,2014-09-18,424.440002
2,2014-09-19,394.795990
3,2014-09-20,408.903992
4,2014-09-21,398.821014
...,...,...
2708,2022-02-15,44575.203125
2709,2022-02-16,43961.859375
2710,2022-02-17,40538.011719
2711,2022-02-18,40030.976563


In [5]:
# closedf
# closedf = closedf[closedf['Date'] > '2021-02-19']

closedf = closedf[closedf['Date'] > '2014-09-17']
closedf = closedf[closedf['Date'] < '2020-08-01']
close_stock = closedf.copy()


In [6]:
close_stock

,Date,Close
1,2014-09-18,424.440002
2,2014-09-19,394.795990
3,2014-09-20,408.903992
4,2014-09-21,398.821014
5,2014-09-22,402.152008
...,...,...
2140,2020-07-27,10990.873047
2141,2020-07-28,10912.823242
2142,2020-07-29,11100.467773
2143,2020-07-30,11111.213867


In [7]:
fig = px.line(closedf, x=closedf['Date'], y=closedf['Close'], labels={'date': 'Date', 'close': 'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Considered Period to predict Bitcoin Close Price', plot_bgcolor='white',
                  font_size=15, font_color='black')
fig.show()

In [8]:
del closedf['Date']
scaler = MinMaxScaler()
closedf = scaler.fit_transform(np.array(closedf).reshape(-1, 1))

closedf

array([[0.01275083],
       [0.0112164 ],
       [0.01194666],
       ...,
       [0.56536035],
       [0.56591659],
       [0.57690316]])

In [9]:
training_size = int(len(closedf) * 0.80)
test_size = len(closedf) - training_size
train_data, test_data = closedf[0: training_size, :], closedf[training_size: len(closedf), :1]

# train_data[:5]
print('train_data shape: ', train_data.shape)
print('test_data shape: ', test_data.shape)

train_data shape:  (1715, 1)
test_data shape:  (429, 1)


In [10]:
def create_XtrainYtrain(dataset, time_step):

  data_X, data_Y = [], []

  for i in range(len(dataset) - time_step - 1):
    a = dataset[i: (i + time_step), 0]
    data_X.append(a)
    data_Y.append(dataset[(i + time_step), 0])

  return np.array(data_X), np.array(data_Y)



In [11]:
time_step = 15
X_train, Y_train = create_XtrainYtrain(train_data, time_step)
X_test, Y_test = create_XtrainYtrain(test_data, time_step)

In [12]:
print('x_train shape:', X_train.shape)
print('y_train shape:', Y_train.shape)
print('x_test shape:', X_test.shape)
print('y_test shape:', Y_test.shape)

x_train shape: (1699, 15)
y_train shape: (1699,)
x_test shape: (413, 15)
y_test shape: (413,)


In [13]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print('X_train.shape: ', X_train.shape)
print('X_test.shape: ', X_test.shape)

X_train.shape:  (1699, 15, 1)
X_test.shape:  (413, 15, 1)


#Model Construction

In [15]:
from tensorflow.keras import optimizers

adam = optimizers.Adam(learning_rate=0.0005)  # Replace 0.001 with your preferred learning rate

model = Sequential()

# Convolutional layers

model.add(Conv1D(filters=5, kernel_size=2, activation='relu', input_shape=(15,1)))
model.add(Conv1D(filters=5, kernel_size=3, activation='relu'))

# Pooling layer
model.add(MaxPooling1D(pool_size=2))

# Flatten layer to transition from convolutional layers to dense layers
# model.add(Flatten())

model.add(LSTM(17, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))


model.compile(loss="mean_squared_error", optimizer=adam)
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test),
                    epochs=200, batch_size=32, verbose=1)

Epoch 1/200
54/54 [==============================] - 2s 7ms/step - loss: 0.0616 - val_loss: 0.1708
Epoch 2/200
54/54 [==============================] - 0s 4ms/step - loss: 0.0397 - val_loss: 0.1105
Epoch 3/200
54/54 [==============================] - 0s 4ms/step - loss: 0.0274 - val_loss: 0.0700
Epoch 4/200
54/54 [==============================] - 0s 4ms/step - loss: 0.0196 - val_loss: 0.0431
Epoch 5/200
54/54 [==============================] - 0s 4ms/step - loss: 0.0153 - val_loss: 0.0262
Epoch 6/200
54/54 [==============================] - 0s 4ms/step - loss: 0.0124 - val_loss: 0.0165
Epoch 7/200
54/54 [==============================] - 0s 3ms/step - loss: 0.0107 - val_loss: 0.0103
Epoch 8/200
54/54 [==============================] - 0s 4ms/step - loss: 0.0096 - val_loss: 0.0071
Epoch 9/200
54/54 [==============================] - 0s 4ms/step - loss: 0.0091 - val_loss: 0.0052
Epoch 10/200
54/54 [==============================] - 0s 5ms/step - loss: 0.0081 - val_loss: 0.0046
Epoch 11/

In [16]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# do the inverse transformation

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

original_ytrain = scaler.inverse_transform(Y_train.reshape(-1, 1))
original_ytest = scaler.inverse_transform(Y_test.reshape(-1, 1))

13/13 [==============================] - 0s 1ms/step


In [17]:
def calculate_mape(actual, predicted) -> float:

    # Convert actual and predicted
    # to numpy array data type if not already
    if not all([isinstance(actual, np.ndarray),
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)

    # Calculate the MAPE value and return
    return round(np.mean(np.abs((
      actual - predicted) / actual)) * 100, 2)


print('Train Data RMSE: ', math.sqrt(mean_squared_error(original_ytrain, train_predict)))
print('Train Data MSE: ', mean_squared_error(original_ytrain, train_predict))
print('Train Data MAE: ', mean_absolute_error(original_ytrain, train_predict))
print('Train MAPE=', calculate_mape(original_ytrain, train_predict))
print('----------------------------------------------------------')
print('Test Data RMSE: ', math.sqrt(mean_squared_error(original_ytest, test_predict)))
print('Test Data MSE: ', mean_squared_error(original_ytest, test_predict))
print('Test Data MAE: ', mean_absolute_error(original_ytest, test_predict))
print('Test MAPE=', calculate_mape(original_ytest, test_predict))


Train Data RMSE:  682.6283524539965
Train Data MSE:  465981.4675740577
Train Data MAE:  510.0639252927066
Train MAPE= 89.94
----------------------------------------------------------
Test Data RMSE:  580.1326932158672
Test Data MSE:  336553.9417378955
Test Data MAE:  396.0033233915708
Test MAPE= 4.53


In [18]:
# shift train predictions for plotting
look_back = time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back: len(train_predict) + look_back, :] = train_predict
print("Train Predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict) + (look_back * 2) + 1: len(closedf)-1, :] = test_predict
print("Test Predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price', 'Train Predicted close price',  'Test Predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                       'train_predicted_close': trainPredictPlot.reshape(1, -1)[0].tolist(),
                       'test_predicted_close': testPredictPlot.reshape(1, -1)[0].tolist()})

fig = px.line(plotdf, x=plotdf['date'], y=[plotdf['original_close'], plotdf['train_predicted_close'], plotdf['test_predicted_close']],
              labels={'value': 'stock price', 'date': 'Data'})

fig.update_layout(title_text='Comparison between original close price vs predicted close price', plot_bgcolor='white', font_size=15,
                  font_color='black', legend_title_text='Close price')

fig.for_each_trace(lambda t: t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


Train Predicted data:  (2144, 1)
Test Predicted data:  (2144, 1)
